In [2]:
%matplotlib inline
import lsst.daf.persistence as dafPersist
import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord
import lsst.afw.image as afwImage
import numpy as np
import matplotlib
import matplotlib.pylab as plt

from lsst.meas.algorithms import LoadIndexedReferenceObjectsTask

#setup butler - Replace path name by something suitable for you
butler = dafPersist.Butler("/sps/lsst/users/lsstprod/hsc/SXDS/input")

refConfig = LoadIndexedReferenceObjectsTask.ConfigClass()
refConfig.ref_dataset_name = 'pan-starrs'
#refConfig.ref_dataset_name = 'gaia'
refTask = LoadIndexedReferenceObjectsTask(butler, config=refConfig)

filt = 'g'
#filt = 'phot_g_mean_mag'
flux = filt + '_flux'

center = afwCoord.Coord('2:18:00', '-5:00:00')
print(center)
radius = 0.010
skyCircle = refTask.loadSkyCircle(center,
                                        afwGeom.Angle(radius, afwGeom.radians), filt)

refCat = skyCircle.refCat.copy(deep=True).asAstropy()

Coord(34.5000000, -5.0000000, 2000.00)


In [2]:
refCat

id,coord_ra,coord_dec,parent,g_flux,r_flux,i_flux,z_flux,y_flux,i_fluxSigma,y_fluxSigma,r_fluxSigma,z_fluxSigma,g_fluxSigma,coord_ra_err,coord_dec_err,epoch,pm_ra,pm_dec,pm_ra_err,pm_dec_err,centroid_x,centroid_y,hasCentroid
,rad,rad,,,,,,,,,,,,,,,,,,,,,
int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,int64,float32,float32,float32,float32,float64,float64,bool
102010343486462892,0.599499154812,-0.0870841698701,0,1.84560246907e-06,2.28611739317e-07,2.4730095447e-06,3.92359606849e-06,8.13839506009e-06,7.30418321382e-07,9.15200962481e-06,3.83430233342e-07,1.32616298743e-06,2.90571392725e-07,0.0,0.0,1328334208,0.0,0.0,0.0,0.0,nan,nan,False
102010343490994670,0.599506624954,-0.0870606467991,0,4.83877420265e-06,6.34796197119e-06,6.21805475021e-06,9.3222479336e-06,1.23368627101e-05,3.15262411732e-07,2.03300919566e-06,7.01367753209e-07,1.35035122639e-06,3.394666237e-07,0.0,0.0,1344082432,0.127443,-0.0435672,0.14006,0.0741894,nan,nan,False
102010343516900982,0.599550377238,-0.0871123949903,0,2.86495696855e-06,8.48813124321e-06,1.07354662759e-05,1.51237318278e-05,1.43542210935e-05,4.39615064352e-07,2.0131835754e-06,3.9348702785e-07,1.11329427455e-06,2.21747214368e-07,0.0,0.0,1365968384,0.0308939,0.00328479,0.0258905,0.0210772,nan,nan,False
102010343553042506,0.599613346841,-0.0870910385568,0,0.000109954649815,0.000154573543114,0.00019433198031,0.000204003954423,0.000203477364266,2.75260822491e-06,5.6540411606e-06,1.56513669936e-05,1.95875577447e-06,1.67837117715e-06,0.0,0.0,1358547968,0.0328359,0.00502027,0.020732,0.00908924,nan,nan,False
102000343554067807,0.599615225459,-0.0871610168209,0,2.97152655548e-06,4.99489624417e-06,6.38470100967e-06,7.40865880289e-06,1.27389630507e-05,4.25506840429e-07,2.08697065318e-06,2.41949777546e-07,7.46703619825e-07,4.51339047913e-07,0.0,0.0,1379498112,0.0462198,-0.0787203,0.0615441,0.0615441,nan,nan,False
102000343555682370,0.599617896905,-0.0872384214165,0,2.07521657103e-06,1.82036967544e-06,3.37114738613e-06,3.63252320312e-06,4.46829471912e-06,4.65756045287e-07,5.91462412558e-06,1.62857588748e-07,6.07644608408e-07,3.42090515915e-07,0.0,0.0,1350212736,0.0,0.0,0.0,0.0,nan,nan,False
102010343563323137,0.599631927829,-0.0870809625223,0,0.012011311017,0.0163842327893,0.0180645026267,0.0191254429519,0.0195187740028,0.00998281314969,6.29578426015e-05,0.00905426219106,1.50719970406e-05,6.37038392597e-05,0.0,0.0,1319358208,0.022022,0.00984402,0.00411038,0.00475597,nan,nan,False
102000343587846009,0.599674218694,-0.087185713239,0,2.23074425776e-06,6.05010836807e-06,8.19891647552e-06,1.0027620192e-05,9.89704130916e-06,2.82353767034e-07,1.80922120308e-06,3.66595429568e-07,7.63664672832e-07,3.4094071566e-07,0.0,0.0,1369053824,0.0846619,-0.0628777,0.0351263,0.0401966,nan,nan,False


In [7]:
def flToMag(flux):
    return -2.5 * np.log10(flux/3631.)

def flToMagErr (flux, fluxErr):
    return np.abs(fluxErr / (-0.4 * flux *np.log(10)))

In [8]:
flToMagErr(refCat[flux], refCat[flux + 'Sigma'])

0.0206571987814
0.00956059634505
0.148223795127
0.240897135938
0.0141438778288
0.0340419833327
0.00184862105133
0.167518658488
0.0681929830223
0.206876928122
0.127779938592


In [9]:
flToMag(refCat[flux])

20.3080033934
19.0196287649
22.1845221555
23.4883571125
20.1379299297
22.8453802884
15.1503193122
22.394483097
21.5339278989
22.6307702146
22.1076345799


In [ ]:
font = {'family' : 'serif',
        'color'  : 'darkred',
        'weight' : 'bold',
        'size'   : 25,
        }

plt.rcParams['axes.linewidth'] = 1.5 #set the value globally

f1 = (refCat['r_flux'] > 0) & (refCat['r_fluxSigma'] > 0.) & (refCat['r_flux']/refCat['r_fluxSigma'] > 1.)

fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(18, 8))
ax0.hist(flToMag(refCat['r_flux'][f1]), bins=80)
ax1.hist(flToMagErr(refCat['r_flux'][f1], refCat['r_fluxSigma'][f1]), bins=80)

fig, (ax2) = plt.subplots(ncols=1, figsize=(18, 8))
ax2.scatter(flToMag(refCat['r_flux'][f1]), flToMagErr(refCat['r_flux'][f1], refCat['r_fluxSigma'][f1]), s=0.2)
ax2.set_xlabel('Magnitude', fontsize=15)
ax2.set_ylabel('$\sigma$(Mag)', fontsize=15)
ax2.tick_params(labelsize=20)

In [ ]:
refCat

In [ ]:
f2 = (refCat['r_flux'] > 0) & (refCat['r_fluxSigma'] > 0.) & (refCat['r_flux']/refCat['r_fluxSigma'] > 5.) 
f2 &= (flToMag(refCat['r_flux']) < 22.5)
fig, (ax0) = plt.subplots(ncols=1, figsize=(10, 10))
ax0.scatter(refCat['coord_ra'], refCat['coord_dec'], s=0.1)
ax0.set_xlabel('RA', fontsize=20)
ax0.set_ylabel('DEC', fontsize=20)
ax0.tick_params(labelsize=20)
fig, (ax1) = plt.subplots(ncols=1, figsize=(10, 10))
ax1.scatter(refCat['coord_ra'][f1], refCat['coord_dec'][f1], s=0.1)
ax1.set_xlabel('RA', fontsize=20)
ax1.set_ylabel('DEC', fontsize=20)
ax1.tick_params(labelsize=20)

In [ ]:
f3 = (refCat['r_flux'] > 0) & (refCat['r_fluxSigma'] > 0.) & (refCat['r_flux']/refCat['r_fluxSigma'] > 0.) 
f3 &= (flToMag(refCat['r_flux']) > 23.2)
fig, (ax0) = plt.subplots(ncols=1, figsize=(10, 10))
ax0.scatter(refCat['coord_ra'][f3], refCat['coord_dec'][f3], s=0.1)
ax0.set_xlabel('RA', fontsize=15)
ax0.set_ylabel('DEC', fontsize=15)
ax0.tick_params(labelsize=15)

In [ ]:
f4 = (f1) & (refCat['coord_ra_err'] > 0.)
fig, (ax0) = plt.subplots(ncols=1, figsize=(10, 10))
ax0.hist(refCat['coord_ra_err'][f4], bins=80)

In [ ]:
        applyColorTerms = self.config.applyColorTerms
        applyCTReason = "config.applyColorTerms is %s" % (self.config.applyColorTerms,)
        if self.config.applyColorTerms is None:
            # apply color terms if color term data is available and photoCatName specified
            ctDataAvail = len(self.config.colorterms.data) > 0
            photoCatSpecified = self.config.photoCatName is not None
            applyCTReason += " and data %s available" % ("is" if ctDataAvail else "is not")
            applyCTReason += " and photoRefCat %s None" % ("is not" if photoCatSpecified else "is")
            applyColorTerms = ctDataAvail and photoCatSpecified

        if applyColorTerms:
            self.log.info("Applying color terms for filterName=%r, config.photoCatName=%s because %s",
                          filterName, self.config.photoCatName, applyCTReason)
            ct = self.config.colorterms.getColorterm(
                filterName=filterName, photoCatName=self.config.photoCatName, doRaise=True)
        else:
            self.log.info("Not applying color terms because %s", applyCTReason)
            ct = None

        if ct:                          # we have a color term to worry about
            fluxFieldList = [getRefFluxField(refSchema, filt) for filt in (ct.primary, ct.secondary)]
            missingFluxFieldList = []
            for fluxField in fluxFieldList:
                try:
                    refSchema.find(fluxField).key
                except KeyError:
                    missingFluxFieldList.append(fluxField)

            if missingFluxFieldList:
                self.log.warn("Source catalog does not have fluxes for %s; ignoring color terms",
                              " ".join(missingFluxFieldList))
                ct = None

        if not ct:
            fluxFieldList = [getRefFluxField(refSchema, filterName)]

        refFluxArrList = []  # list of ref arrays, one per flux field
        refFluxErrArrList = []  # list of ref flux arrays, one per flux field
        for fluxField in fluxFieldList:
            fluxKey = refSchema.find(fluxField).key
            refFluxArr = np.array([m.first.get(fluxKey) for m in matches])
            try:
                fluxErrKey = refSchema.find(fluxField + "Sigma").key
                refFluxErrArr = np.array([m.first.get(fluxErrKey) for m in matches])
            except KeyError:
                # Reference catalogue may not have flux uncertainties; HACK
                self.log.warn("Reference catalog does not have flux uncertainties for %s; using sqrt(flux).",
                              fluxField)
                refFluxErrArr = np.sqrt(refFluxArr)

            refFluxArrList.append(refFluxArr)
            refFluxErrArrList.append(refFluxErrArr)

        if ct:                          # we have a color term to worry about
            refMagArr1 = np.array([abMagFromFlux(rf1) for rf1 in refFluxArrList[0]])  # primary
            refMagArr2 = np.array([abMagFromFlux(rf2) for rf2 in refFluxArrList[1]])  # secondary

            refMagArr = ct.transformMags(refMagArr1, refMagArr2)
            refFluxErrArr = ct.propagateFluxErrors(refFluxErrArrList[0], refFluxErrArrList[1])
        else:
            refMagArr = np.array([abMagFromFlux(rf) for rf in refFluxArrList[0]])

        srcMagArr = np.array([abMagFromFlux(sf) for sf in srcFluxArr])

        # Fitting with error bars in both axes is hard
        # for now ignore reference flux error, but ticket DM-2308 is a request for a better solution
        magErrArr = np.array([abMagErrFromFluxErr(fe, sf) for fe, sf in zip(srcFluxErrArr, srcFluxArr)])
        if self.config.magErrFloor != 0.0:
            magErrArr = (magErrArr**2 + self.config.magErrFloor**2)**0.5

        srcMagErrArr = np.array([abMagErrFromFluxErr(sfe, sf) for sfe, sf in zip(srcFluxErrArr, srcFluxArr)])
        refMagErrArr = np.array([abMagErrFromFluxErr(rfe, rf) for rfe, rf in zip(refFluxErrArr, refFluxArr)])

        return pipeBase.Struct(
            srcMag=srcMagArr,
            refMag=refMagArr,
            magErr=magErrArr,
            srcMagErr=srcMagErrArr,
            refMagErr=refMagErrArr,
            refFluxFieldList=fluxFieldList,
        )

In [3]:
from lsst.pipe.tasks.colorterms import ColortermLibrary
from lsst.afw.image import abMagFromFlux, abMagErrFromFluxErr, fluxFromABMag, Calib

In [4]:
import os
from lsst.utils import getPackageDir

hscConfigDir = os.path.join(getPackageDir("obs_subaru"), "config", "hsc")
colorterms = ColortermLibrary()
colorterms.load(os.path.join(hscConfigDir, 'colorterms.py'))

photoCatName = 'ps1*'

ct = colorterms.getColorterm(
            filterName='g', photoCatName=photoCatName, doRaise=True)

In [5]:
ct.

lsst.pipe.tasks.colorterms.Colorterm(primary='g', secondary='r', c0=0.00730066, c1=0.06508481, c2=-0.0151057)

In [6]:
ct.secondary

'r'

In [20]:
mag = abMagFromFlux(refCat[2001]['g_flux'])
err = abMagErrFromFluxErr(refCat[2001]['g_flux'], refCat[2001]['g_fluxSigma'])
err2 = abMag

mag, err

(23.84107582255976, 5.108196971524156)

In [16]:
abMagErrFromFluxErr

<function lsst.afw.image.calib.PyCapsule.abMagErrFromFluxErr>

In [27]:
a = np.arange(10)
b = np.arange(10)*2
a, b

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18]))

In [25]:
c =[i*j for i,j in zip(a,b)]

In [26]:
c

[0, 2, 8, 18, 32, 50, 72, 98, 128, 162]

In [30]:
filters = ['r', 'i']
for f in filters :
    print(f)

r
i


In [31]:
test = {}
test['g'] = a
test['r'] = b
test

{'g': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 'r': array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])}